In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection,\
linear_model, metrics, pipeline

In [2]:
df = pd.read_csv("/data/credit-default.csv")
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [3]:
df.default.value_counts()

1    700
2    300
Name: default, dtype: int64

In [4]:
target = "default"
X = df.drop(columns=[target])
X = pd.get_dummies(X, drop_first=True)
columns = X.columns

X = X.values.astype("float")

y = preprocessing.LabelEncoder().fit_transform(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                           test_size = 0.3, random_state = 147595)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1
                                    , include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LogisticRegression(random_state=1))
])


pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [7]:
metrics.confusion_matrix(y_train, y_train_pred)

array([[442,  50],
       [ 80, 128]])

In [8]:
(442 + 128)/len(X_train)

0.8142857142857143

In [9]:
metrics.accuracy_score(y_train, y_train_pred)

0.8142857142857143

In [11]:
pd.Series(y_train).value_counts()/len(y_train)

0    0.702857
1    0.297143
dtype: float64

In [12]:
metrics.accuracy_score(y_test, y_test_pred)

0.69

In [13]:
pd.Series(y_test).value_counts()/len(y_test)

0    0.693333
1    0.306667
dtype: float64